In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import seaborn as sns

resultado_anterior = 0
maximo = 0

# Carga de datos

In [2]:
auctions = pd.read_csv('auctions.csv')
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce", low_memory=False)

In [3]:
clicks_dtypes = {'advertiser': np.int8, 'action_id': np.float16, 'source_id': np.int8, 'country_code': 'category',
                 'latitude': np.float16, 'longitude': np.float16, 'carrier_id': np.float16, 'brand': np.float16,
                 'touchX': np.float16, 'touchY': np.float16}
clicks = pd.read_csv('clicks.csv')
clicks["created"] =  pd.to_datetime(clicks["created"], errors = "coerce", low_memory=False) #parseo de fechas

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
events = pd.read_csv('events.csv', low_memory=False)
events["date"] =  pd.to_datetime(events["date"], errors = "coerce")

In [5]:
installs = pd.read_csv('installs.csv')
installs["created"] =  pd.to_datetime(installs["created"], errors = "coerce", low_memory=False) 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Features

### Creación del df para predecir

In [90]:
df_features = pd.DataFrame(auctions["device_id"].unique())
df_features.columns = ['ref_hash']
df_features = df_features.set_index('ref_hash')

In [91]:
def agregar_feature(df_features, df_to_join, nombre, fill=0):
    df_features = df_features.join(df_to_join, on="ref_hash", how="inner")
    return df_features

## feature1

In [92]:
installs["tiene_installs"] = 1
agregar_feature(df_features, installs, "tiene_installs", 0)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,tiene_installs
ref_hash,,,,,,,,,,,,,,,,,,,


In [43]:
df_test = installs.loc[:,["ref_hash"]]
df_test["tiene_installs"] = df_test["ref_hash"].isin(installs["ref_hash"])).astype(int)

serie = pd.Series(df_features.index.isin(installs["ref_hash"])).astype(int)
df_to_join = pd.DataFrame( {"tiene_installs": serie}, index= df_features.index ) 
df_to_join
#agregar_feature(df_features, df_to_join)

SyntaxError: invalid syntax (<ipython-input-43-30d64dc39ac8>, line 2)

In [81]:
pd.Series(df_features.index.isin(installs["ref_hash"])).value_counts()

True     241835
False    233503
dtype: int64

# Entrenamiento

In [12]:
entrenar_modelo({})

In [11]:
def guardar_submit(params, result):
    tiempo = "time"
    with open("historial_submits.txt","a+") as f:
        f.write("\n"+tiempo+"|"+params+"|")

In [3]:
def entrenar_modelo(params):
    result = None#entrenar(params)
    guardar_submit(params, result)
    

In [ ]:
def modelfit(model, features, labels, usarCrossValidation=True, cv_folds=5, early_stopping_rounds=50):
    if usarCrossValidation:
        parametros = model.get_xgb_params()
        dmatrix = xgb.DMatrix(features.values, label=labels.values)
        cvresult = xgb.cv(parametros, dmatrix, num_boost_round=model.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='auc', #verbose_eval=True,
                          early_stopping_rounds=early_stopping_rounds)
        model.set_params(n_estimators=cvresult.shape[0])
    
    model.fit(features, labels,eval_metric='auc')

In [ ]:
X, y = dfPredictorLocal.iloc[:,:-1], dfPredictorLocal.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

modelo = xgb.XGBRegressor(
 #booster='dart',
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=8,
 gamma=0.22,
 subsample=1,
 colsample_bytree=1,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=18.8,

 #sample_type='weighted',
 #rate_drop=0.1,
 #skip_dropout=0.5,
 random_state=272)

modelfit(modelo, X_train, y_train, early_stopping_rounds=30)

predsa = modelo.predict(X_test)
resultado = roc_auc_score(y_test, predsa)
print(resultado)
print("Vario: {} con respecto al anterior".format(resultado-resultado_anterior))
if (maximo == resultado):
    print(" Maximo igualado")
else:
    print("Maximo anterior {}".format(maximo))
if resultado>maximo:
    print("Nuevo MAXIMO : {}".format(resultado-maximo))
    maximo = resultado
resultado_anterior = resultado